In [1]:
#Download spacy library
!python -m spacy download fr_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [2]:
#Importing required packages
import pandas as pd
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import numpy as np
import gensim
from sklearn import datasets
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.utils.multiclass import unique_labels
import string

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#Import data (cognates and train data Aicrowd)

Sentences = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Dataset_Aicrowd/22fdbaea-415d-4685-8132-1916959ca359_train.csv"
Sentences = pd.read_csv(Sentences)
Sentences.columns = ['Text','Difficulty']

Cognates = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates/Cognates.csv"
Cognates = pd.read_csv(Cognates)

CognatesGlossary = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates/glossaryCognates.csv"
CognatesGlossary = pd.read_csv(CognatesGlossary)

In [5]:
#Preprocess the punctuation and digits
def remove_punctuations(text):
    punctuations = '\!"#$%&()*+,./:;<=>?@[\]_{|}~'
    for punctuation in punctuations:
        text = text.replace(punctuation, '')
        text = text.replace('-', ' ')
    return text

def remove_digits(text):
    for digits in string.digits:
        text = text.replace(digits, '')
    return text
Sentences.columns = ['Text','Difficulty']
Sentences["Text"] = Sentences['Text'].apply(remove_digits)

Sentences["Text"] = Sentences['Text'].apply(remove_punctuations)

In [6]:
Cognates.head()

,Cognates
0,abattoir
1,abdication
2,abdominal
3,aberrant
4,aberration


In [7]:
CognatesGlossary.head()

,CognatesEN,CognatesFR
0,absolutely,absolument
1,abundance,abondance
2,abuse,abuser
3,accelerate (to),accélérer
4,accentuate (to),accentuer


In [8]:
#Create a cognates list and delete duplicates
ListCo = []
for cognate  in CognatesGlossary.CognatesFR:
  ListCo.append(cognate)
for cognate  in Cognates.Cognates:
  ListCo.append(cognate)
ListCognates = list(set(ListCo))
print(ListCognates)
print(len(ListCognates))

['placement', 'impertinence', 'air', 'phylum', 'correction', 'lobe', 'paraphrase', 'trauma', 'navigation', 'absolution', 'expérience', 'satisfaction', 'talc', 'indépendance', 'vestibule', 'conversion', 'cassette', 'investiture', 'justifiable', 'quiche', 'surplus', 'isotope\xa0', 'veto', 'desperado', 'narration', 'vaudeville', 'susceptible', 'gorille', 'constriction', 'magnifique', 'microphone', 'ersatz', 'ricochet', 'ramification', 'castration', 'kilt', 'stagnant', 'vivisection', 'royal', 'crocodile', 'addition', 'canyon', 'colonel', 'restaurant', 'accélérer', 'barge', 'catalogue', 'usuelle', 'digestible', 'combat', 'terrain', 'libido', 'actuaire', 'nerveux', 'koala', 'caméra', 'suspicion', 'smog', 'mode', 'marionnette', 'amusement', 'barrage', 'efficient', 'magnitude', 'sardine', 'généreux', 'civiliser', 'intermission', 'sanction', 'indivisible', 'introduire', 'affectation', 'pigment', 'aberration', 'contraception', 'dictatorial', 'imitable', 'prototype', 'horticulture', 'urine', 'col

In [9]:
pd.set_option("max_colwidth", None)
Sentences.head(10)

,Text,Difficulty
0,C'est pour quand,A1
1,Je pense que c'est bon,A1
2,C'est pas mal,A1
3,Qu'est ce que vous faites,A1
4,C'est bien,A1
5,Ce n'est pas juste,A1
6,Je ne veux pas,A1
7,Je ne sais pas,A1
8,Plus que ça,A1
9,Non je ne le trouve pas,A1


In [10]:
#Calculating base rate (perfect base rate with 6 features is 0.16)
max(len(Sentences[Sentences["Difficulty"] =='A1']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'A2']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'B1']) / len(Sentences) , len(Sentences[Sentences["Difficulty"] == 'B2']) / len(Sentences),len(Sentences[Sentences["Difficulty"] == 'C1']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'C2']) / len(Sentences))

0.16666666666666666

In [11]:
#Counting targets values 
display(Sentences.Difficulty.value_counts())

A2    800
B1    800
C2    800
A1    800
B2    800
C1    800
Name: Difficulty, dtype: int64

In [12]:
#Counting targets values 
indexMax = (Sentences.shape[0] -1)
Sentences.shape[0]

4800

In [13]:
#Create a new dataset and add a new columns for Cognates
df = Sentences
df = df.assign(NumberOfCognate = 0)
df = df.assign(Cognates = "")
df.head()

,Text,Difficulty,NumberOfCognate,Cognates
0,C'est pour quand,A1,0,
1,Je pense que c'est bon,A1,0,
2,C'est pas mal,A1,0,
3,Qu'est ce que vous faites,A1,0,
4,C'est bien,A1,0,


In [14]:
#Tokenizer sentences with spacy for French language
sp = spacy.load('fr_core_news_sm')
index = 0
data = list(zip(Sentences["Text"],Sentences["Difficulty"]))
datawords = []
for t, label in data:
  list_of_strings = sp(t)
  token = []
  cognates =""
  score=0
  for i in list_of_strings:
    token.append(i.text)
    # Duplicate the Cognates on the sentences (add weight) and count the Cognates per sentences
    if str(i.text) in ListCognates:
      score+=1 
      token.append(i.text)
      cognates = cognates + " " + str(i.text)
  if index <=indexMax:
    df.at[index,'Cognates']= cognates
    df.at[index,'NumberOfCognate']= int(score)
    index += 1
  datawords.append([token, label])

In [15]:
# Data set with the identified Cognates for each Sentences
df.head(-1)

,Text,Difficulty,NumberOfCognate,Cognates
0,C'est pour quand,A1,0,
1,Je pense que c'est bon,A1,0,
2,C'est pas mal,A1,0,
3,Qu'est ce que vous faites,A1,0,
4,C'est bien,A1,0,
...,...,...,...,...
4794,Le jardin commença à fleurir après qu'il se fut mis à faire chaud,C2,1,jardin
4795,Le problème va du garçon de café tentant inintelligiblement de communiquer avec des touristes médusés jusque comme on l'a vu aux niveaux les plus élevés de la société française,C2,2,problème plus
4796,Les lois de programmation mentionnées à l'antépénultième alinéa de l'article de la Constitution ont pour objet de déterminer les objectifs de l'action de l'État,C2,4,article objet déterminer action
4797,La conscience malheureuse c'est donc le mauvais infini tel que la philosophie de Fichte amène à le concevoir,C2,1,conscience


In [16]:
# Cognates Analysis
cognatesSum = df.NumberOfCognate.sum()
print("We found in total : ", cognatesSum, "cognates.")
print("The sentence with the most cognates is :")
df.sort_values(by=['NumberOfCognate'],ascending=False).head(1)

We found in total :  3798 cognates.
The sentence with the most cognates is :


,Text,Difficulty,NumberOfCognate,Cognates
4383,Ceci dit il reste qu'entre l'ovule et le spermatozoïde on observe des différences secondaires des plus intéressantes la singularité essentielle de l'ovule c'est qu'il est chargé de matériaux destinés à nourrir et à protéger l'embryon il accumule des réserves aux dépens desquelles le foetus édifiera ses tissus réserves qui sont non une substance vivante mais une matière inerte il en résulte qu'il présente une forme massive sphérique ou ellipsoïdale et qu'il est relativement volumineux on sait quelles dimensions atteint l'oeuf de l'oiseau chez la femme l'ovule mesure mm de diamètre tandis que dans le sperme humain on trouve spermatozoïdes par millimètre cube la masse du spermatozoïde est extrêmement réduite il a une queue filiforme une petite tête allongée aucune substance étrangère ne l'alourdit il est tout entier vie cette structure le voue à la mobilité au lieu que l'ovule où se trouve engrangé l'avenir du foetus est un élément fixe enfermé dans l'organisme femelle ou suspendu dans un milieu extérieur il attend passivement la fécondation c'est le gamète mâle qui va à sa recherche le spermatozoïde est toujours une cellule nue l'ovule est selon les espèces protégé ou non par une membrane mais en tout cas dès que le spermatozoïde entre en contact avec lui il le bouscule le fait osciller et s'infiltre en lui le gamète mâle abandonne sa queue sa tête se gonfle et d'un mouvement tournant gagne le noyau pendant ce temps l'oeuf forme aussitôt une membrane qui le ferme aux autres spermatozoïdes,C2,10,plus substance humain cube substance structure organisme membrane contact membrane


In [17]:
# Data set for the first iteration : with the number of cognates for each sentences.
FirstIteration = pd.DataFrame()
FirstIteration['Text'] = df['Text'].astype('string') + ' ' +df['NumberOfCognate'].astype('string')
FirstIteration['label'] = df['Difficulty']
FirstIteration.head(-1)

,Text,label
0,C'est pour quand 0,A1
1,Je pense que c'est bon 0,A1
2,C'est pas mal 0,A1
3,Qu'est ce que vous faites 0,A1
4,C'est bien 0,A1
...,...,...
4794,Le jardin commença à fleurir après qu'il se fut mis à faire chaud 1,C2
4795,Le problème va du garçon de café tentant inintelligiblement de communiquer avec des touristes médusés jusque comme on l'a vu aux niveaux les plus élevés de la société française 2,C2
4796,Les lois de programmation mentionnées à l'antépénultième alinéa de l'article de la Constitution ont pour objet de déterminer les objectifs de l'action de l'État 4,C2
4797,La conscience malheureuse c'est donc le mauvais infini tel que la philosophie de Fichte amène à le concevoir 1,C2


In [18]:
# Data set for second iteration : with tokenized sentences and with duplicate congantes
SecondIteration = pd.DataFrame(datawords, columns=["Text", "Label"])
SecondIteration.head(-1)

,Text,Label
0,"[C', est, pour, quand]",A1
1,"[Je, pense, que, c', est, bon]",A1
2,"[C', est, pas, mal]",A1
3,"[Qu', est, ce, que, vous, faites]",A1
4,"[C', est, bien]",A1
...,...,...
4794,"[Le, jardin, jardin, commença, à, fleurir, après, qu', il, se, fut, mis, à, faire, chaud]",C2
4795,"[Le, problème, problème, va, du, garçon, de, café, tentant, inintelligiblement, de, communiquer, avec, des, touristes, médusés, jusque, comme, on, l', a, vu, aux, niveaux, les, plus, plus, élevés, de, la, société, française]",C2
4796,"[Les, lois, de, programmation, mentionnées, à, l', antépénultième, alinéa, de, l', article, article, , de, la, Constitution, ont, pour, objet, objet, de, déterminer, déterminer, les, objectifs, de, l', action, action, de, l', État]",C2
4797,"[La, conscience, conscience, malheureuse, c', est, donc, le, mauvais, infini, tel, que, la, philosophie, de, Fichte, amène, à, le, concevoir]",C2


In [19]:
# Data set for the third iteration : 
FinalData = pd.DataFrame()
FinalData['Text'] = df['Text'].astype('string') + df['Cognates'].astype('string')
FinalData['label'] = df['Difficulty']
FinalData.head(-1)

,Text,label
0,C'est pour quand,A1
1,Je pense que c'est bon,A1
2,C'est pas mal,A1
3,Qu'est ce que vous faites,A1
4,C'est bien,A1
...,...,...
4794,Le jardin commença à fleurir après qu'il se fut mis à faire chaud jardin,C2
4795,Le problème va du garçon de café tentant inintelligiblement de communiquer avec des touristes médusés jusque comme on l'a vu aux niveaux les plus élevés de la société française problème plus,C2
4796,Les lois de programmation mentionnées à l'antépénultième alinéa de l'article de la Constitution ont pour objet de déterminer les objectifs de l'action de l'État article objet déterminer action,C2
4797,La conscience malheureuse c'est donc le mauvais infini tel que la philosophie de Fichte amène à le concevoir conscience,C2


In [20]:
display(FinalData.shape, FinalData.info(), FinalData.describe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    4800 non-null   string
 1   label   4800 non-null   object
dtypes: object(1), string(1)
memory usage: 75.1+ KB


(4800, 2)

None

<bound method NDFrame.describe of                                                                                                                                                                                                                                    Text label
0                                                                                                                                                                                                                     C'est pour quand     A1
1                                                                                                                                                                                                                Je pense que c'est bon    A1
2                                                                                                                                                                                                                         C'est pas mal    A1
3             

In [21]:
FirstIteration.to_csv('FirstIteration.csv', encoding='utf-8-sig', index=False)
SecondIteration.to_csv('SecondIteration.csv', encoding='utf-8-sig', index=False)
FinalData.to_csv('ThirdIteration.csv', encoding='utf-8-sig', index=False)